# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal



fs = 125  
minBPM = 40  
maxBPM = 240  
window_length = 8 * fs  
window_shift = 2 * fs  


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def bandpass_filter(signal, fs=125, order=3, window=[40/60, 240/60]):
    """filter the signal between 40 and 240 BPM

    Args:
        signal: input signal
        fs: sampling freq

    Returns:
        new signal  withing the range of the desired frequencies
    """
    
    b, a = scipy.signal.butter(order, window, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)


def fft(signal, fs = 125):
    """
    computes the forier transform and ft frequency of the passed signal
    
    Args:
        signal: input signal
        fs: Hz of input signal

    Returns:
        fft: fourier transform
        freqs: fourier transformed frequencies
    """
    n_samples = len(signal)  # No zero padding
    freqs = np.fft.rfftfreq(n_samples, 1/fs)
    fft = np.abs(np.fft.rfft(signal, n_samples))
    return freqs, fft

def calculate_magnitude(x, y, z):
    """
    Calculate the magnitude for the signal

    Args:
        x: x-axis
        y: y-axis
        z: z-axis

    Returns:
        Signal magnitude value
    """
    return np.sqrt(x**2 + y**2 + z**2) 


def RunPulseRateAlgorithm(data_fl, ref_fl):
    # load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # bandpass filter the signals
    ppg = bandpass_filter(ppg, fs)
    accx = bandpass_filter(accx, fs)
    accy = bandpass_filter(accy, fs)
    accz = bandpass_filter(accz, fs)
    
    # Consider only magnitude of acceleration
    acc = calculate_magnitude(accx, accy, accz)
    
    # loading the reference file
    ground_truth = sp.io.loadmat(ref_fl)['BPM0'].reshape(-1)
   
    bpm_pred, confidence = [], []

    # Compute both bpm prediction and confidence value    
    for i in range(0, len(ppg) - window_length + 1, window_shift):
        ppg_window = ppg[i:i+window_length]

        # aggregate accelerometer data into single signal to get the acc window
        ppg_window = ppg[i:i+window_length]
        acc_window = acc[i:i+window_length]
        
        # Perform Fourier transform
        ppg_freqs, ppg_fft = fft(ppg_window)
        acc_freqs, acc_fft = fft(acc_window)
        
        # Remove unwanted frequencies 

        # Filter data
        ppg_fft[ppg_freqs <= 40/60.0] = 0.0
        ppg_fft[ppg_freqs >= 240/60.0] = 0.0
        acc_fft[acc_freqs <= 40/60.0] = 0.0
        acc_fft[acc_freqs >= 240/60.0] = 0.0
    
        # Find peaks
        ppg_peaks = sp.signal.find_peaks(ppg_fft, height=2000)[0]
        acc_peaks = sp.signal.find_peaks(acc_fft, height=None)[0]
        
        # Pick frequency with largerst FFT coefficient
        ppg_max = ppg_freqs[np.argmax(ppg_fft)]
        acc_max = acc_freqs[np.argmax(acc_fft)]
        
        if(acc_max != ppg_max):
            est_bpm = ppg_max * 60
                
        # Append largest BPM and calculated confidence computed         
        bpm_pred.append(est_bpm)
        
        # Calculating Confidence
        conf_wind = (ppg_freqs > ppg_max - 1) & (ppg_freqs < ppg_max + 1)  
        conf = np.sum(ppg_fft[conf_wind]) / np.sum(ppg_fft)

        confidence.append(conf)
        
        
    # Calculate errors
    errors = np.abs(np.subtract(bpm_pred, ground_truth))
        
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.array(errors), np.array(confidence)
